Create DataFrames for each patient to train SVM

In [2]:
import pandas as pd
import numpy as np
import SimpleITK as sitk
import glob
import os
import re

def create_col(p_ID, mode):
    # modalities =  GM, MD, FA, FICVF, ODI
    dir = "OUTPUT/DISCRETE/LESIONS/"
    modes = {'GM': dir + 'GM-LESIONS/',
            'MD': dir + 'MD-LESIONS/',
            'FA': dir + 'FA-LESIONS/',
            'FICVF': dir + 'FICVF-LESIONS/',
            'ODI': dir + 'ODI-LESIONS/'
           }
    # data is sitk image type
    # p_ID is patient ID (string)
    
    file = glob.glob( modes[mode] + "*" + p_ID + "*.nii")
    if not file:
        print(mode + " modality doesn't exist for " + p_ID)
        return [0] * 2122945 # voxels in flattened mri

    img = sitk.ReadImage(file[0])
    arr = sitk.GetArrayFromImage(img)
    arr = arr.flatten()
    return arr
   
gm_lesions_file_list = glob.glob("OUTPUT/DISCRETE/LESIONS/GM-LESIONS/*")
gm_lesions_file_list.sort()
    
patient_ids = []

for file in gm_lesions_file_list:
    patient_ids.append(re.search('.*(D[0-9][0-9][0-9]).*', os.path.basename(file)).group(1))


ALL_FRAMES =[]

for patient in patient_ids:
    df = pd.DataFrame({'GM' : create_col(patient, "GM"),
                        'FA' : create_col(patient, "FA"),
                        'MD' : create_col(patient, "MD"),
                        'FICVF' : create_col(patient, "FICVF"),
                        'ODI' : create_col(patient, "ODI"),
                        'Target' : 1,
                        'Target name' : 'Discrete',
    })
    ALL_FRAMES.append(df)

test = ALL_FRAMES[0]